### [3] 인구 분포 <hr>

모듈 로딩) <hr>

In [6]:
# 모듈 로딩
import pandas as pd
import matplotlib.pyplot as plt
import os

한글 폰트 설정) <hr>

In [7]:
# ============================================================================
# 한글 폰트 설정 - 간단한 방법
# ============================================================================
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

print(f"현재 폰트: {plt.rcParams['font.family']}\n")

현재 폰트: ['Malgun Gothic']



데이터 로드) <hr>

In [8]:
# ============================================================================
# 필요한 데이터 로드
# ============================================================================

# 데이터 로드
population = '../data/raw/population.csv'

df_population = pd.read_csv(population, encoding='utf-8', low_memory=False)

print("=" * 80)
print("시도별 인구 분석")
print("=" * 80)

print(f"\n총 데이터: {len(df_population)}개 행")
print(f"컬럼: {list(df_population.columns)}")
display(df_population.head(3))

시도별 인구 분석

총 데이터: 19개 행
컬럼: ['행정구역별', '2025.06', '2025.06.1', '2025.06.2', '2025.07', '2025.07.1', '2025.07.2', '2025.08', '2025.08.1', '2025.08.2']


,행정구역별,2025.06,2025.06.1,2025.06.2,2025.07,2025.07.1,2025.07.2,2025.08,2025.08.1,2025.08.2
0,행정구역별,계 (명),남 (명),여 (명),계 (명),남 (명),여 (명),계 (명),남 (명),여 (명)
1,전국,51164582,25467115,25697467,51159889,25463429,25696460,51154981,25459789,25695192
2,서울특별시,9325616,4497354,4828262,9323492,4495747,4827745,9321863,4494474,4827389


데이터 전처리) <hr>

In [9]:
# ============================================================================
# 데이터 전처리(필요없는 열 삭제)(['행정구역별', '2025.08']만 남겨둠)
# ============================================================================
# 브랜드명과 지역만 선택하여 지역별로 정렬
df_population = df_population[['행정구역별', '2025.08']]

# ============================================================================
# 행정구역별 컬럼을 인덱스로 변경
# ============================================================================
df_population.set_index('행정구역별', inplace=True)

# ============================================================================
# 행정구역별, 전국 행 삭제
# ============================================================================
df_population.drop(['행정구역별', '전국'], inplace=True)

# ============================================================================
# 컬럼 이름 변경 (행정구역별 → 시도, 2025.08 → 인구)
# ============================================================================
df_population.rename(columns={'행정구역별': '시도', '2025.08': '인구'}, inplace=True)

# ============================================================================
# 저장 경로 설정
# ============================================================================
# processed 폴더(전처리 데이터 저장)
save_path_processed = '../data/processed'
os.makedirs(save_path_processed, exist_ok=True)

# reports 폴더(시도별 인구 현황 저장)
save_path_reports = '../reports/tables'
os.makedirs(save_path_reports, exist_ok=True)

# ============================================================================
# 전처리 데이터 저장
# ============================================================================
csv_file_processed = os.path.join(save_path_processed, 'population_preprocessed.csv')
df_population.to_csv(csv_file_processed, encoding='utf-8-sig')
print(f"✓ 전처리된 데이터 저장: {os.path.abspath(csv_file_processed)}")

# ============================================================================
# 시도별 인구 현황 저장
# ============================================================================
csv_file_stats = os.path.join(save_path_reports, 'population_stats.csv')
df_population.to_csv(csv_file_stats, encoding='utf-8-sig')
print(f"✓ 시도별 인구 현황 저장: {os.path.abspath(csv_file_stats)}\n")

✓ 전처리된 데이터 저장: c:\Users\khw27\바탕 화면\MINI2 Pandas\data\processed\population_preprocessed.csv
✓ 시도별 인구 현황 저장: c:\Users\khw27\바탕 화면\MINI2 Pandas\reports\tables\population_stats.csv



시각화 <hr>

In [10]:
# ============================================================================
# 시각화 (matplotlib)
# ============================================================================

print("\n" + "=" * 80)
print("시각화 생성")
print("=" * 80)


# 1. 시도별 인구 분포 (막대그래프)
fig1, ax1 = plt.subplots(figsize=(8, 6))

# (1) 인구 열을 숫자형으로 변환 (콤마 제거 후 int 변환)
df_population['인구'] = (
    df_population['인구']
    .astype(str)           # 혹시 숫자 외의 타입이 섞여 있을 경우 대비
    .str.replace(',', '', regex=False)  # 천 단위 구분 콤마 제거
    .astype(float)         # 숫자형으로 변환 (float or int 가능)
)

# (2) 오름차순 정렬
df_population = df_population.sort_values(by='인구', ascending=True)

# (3) 가로 막대 그래프 생성
bars = ax1.barh(df_population.index, df_population['인구'], color='skyblue')  

ax1.set_title('시도별 인구 분포', fontsize=12, fontweight='bold')
ax1.set_xlabel('개수')

for bar in bars:
    width = bar.get_width()
    ax1.text(width, bar.get_y() + bar.get_height()/2,
             f'{int(width):,}', ha='left', va='center',
             fontsize=9, fontweight='bold')

fig1.tight_layout()
fig1.savefig('../reports/figures/3_01_df_population_stats.png', dpi=300)
plt.close(fig1)
print("✓ 그래프 저장: reports/figures/3_01_df_population_stats.png")


# 2. 시도별 인구 비율 (막대그래프)
fig2, ax2 = plt.subplots(figsize=(8, 6))

pop_series = df_population['인구']
total_count = pop_series.sum()

df_population_stats_percent = (pop_series / total_count * 100).sort_values(ascending=True)

bars2 = ax2.barh(df_population_stats_percent.index, df_population_stats_percent.values, color='coral')

ax2.set_title('시도별 인구 비율', fontsize=12, fontweight='bold')
ax2.set_xlabel('비율 (%)')

for bar in bars2:
    width = bar.get_width()
    ax2.text(width, bar.get_y() + bar.get_height()/2,
             f'{width:.1f}%', ha='left', va='center',
             fontsize=9, fontweight='bold')

fig2.tight_layout()
fig2.savefig('../reports/figures/3_02_df_population_ratio.png', dpi=300)
plt.close(fig2)
print("✓ 그래프 저장: reports/figures/3_02_df_population_ratio.png")



시각화 생성
✓ 그래프 저장: reports/figures/3_01_df_population_stats.png
✓ 그래프 저장: reports/figures/3_02_df_population_ratio.png
